In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df = pd.read_csv('data/cleaned.csv')

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.tail()

,imdb_title_id,title,year,genre,duration,country,language,director,writer,actors,description,weighted_average_vote,mean_vote,reviews_from_users,reviews_from_critics,females_allages_avg_vote,males_allages_avg_vote,link
85768,tt9906644,Manoharam,2019,"Comedy, Drama",122,India,Malayalam,Anvar Sadik,unavailable,"Vineeth Sreenivasan, Aparna Das, Basil Joseph,...",Manoharan is a poster artist struggling to fin...,6.8,7.2,9.0,1.0,7.3,6.7,https://www.imdb.com/title/tt9906644
85769,tt9908390,Le lion,2020,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,"Alexandre Coquelle, Matthieu Le Naour","Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,5.5,unavailable,4.0,6.0,5.3,https://www.imdb.com/title/tt9908390
85770,tt9911196,De Beentjes van Sint-Hildegard,2020,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,"Radek Bajgar, Herman Finkers","Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,7.9,6.0,4.0,7.6,7.8,https://www.imdb.com/title/tt9911196
85771,tt9914286,Sokagin Çocuklari,2019,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,"Ahmet Faik Akinci, Kasim Uçkan","Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",unavailable,6.4,9.4,unavailable,unavailable,4.0,3.1,https://www.imdb.com/title/tt9914286
85772,tt9914942,La vida sense la Sara Amat,2019,Drama,74,Spain,Catalan,Laura Jou,"Coral Cruz, Pep Puig","Maria Morera Colomer, Biel Rossell Pelfort, Is...","Pep, a 13-year-old boy, is in love with a girl...",6.7,6.8,unavailable,2.0,6.9,6.6,https://www.imdb.com/title/tt9914942


In [4]:
features = ['country', 'language', 'description', 'actors']

In [5]:
def combine_features(row):
    return row['country'] + row['language'] + row['description'] + row['actors']

In [6]:
for feature in features:
    df[feature] = df[feature].fillna('')
    df["combined_features"] = df.apply(combine_features,axis=1)

In [7]:
df1 = df.iloc[:25000, :]

In [8]:
df2 = df.iloc[25001:50000, :]

In [9]:
df3 = df.iloc[50001:75000, :]

In [10]:
df4 = df.iloc[75001:85773, :]

In [11]:
df4

,imdb_title_id,title,year,genre,duration,country,language,director,writer,actors,description,weighted_average_vote,mean_vote,reviews_from_users,reviews_from_critics,females_allages_avg_vote,males_allages_avg_vote,link,combined_features
75001,tt4560436,Red Zone - 22 miglia di fuoco,2018,"Action, Thriller",94,"USA, China, Colombia","English, Russian, Indonesian, Filipino",Peter Berg,"Graham Roland, Lea Carpenter","Mark Wahlberg, Lauren Cohan, Iko Uwais, John M...",A small team of elite American intelligence of...,6.1,6.3,815.0,204.0,6.2,6.1,https://www.imdb.com/title/tt4560436,"USA, China, ColombiaEnglish, Russian, Indonesi..."
75002,tt4560926,Birdland,2018,"Crime, Mystery",89,Canada,English,Peter Lynch,"Lee Gowan, Peter Lynch","Kathleen Munroe, David Alpay, Benjamin Ayres, ...",An ex-cop whose marriage is on the rocks hides...,3.5,3.8,4.0,3.0,4.1,3.4,https://www.imdb.com/title/tt4560926,CanadaEnglishAn ex-cop whose marriage is on th...
75003,tt4561016,The Last Witch,2015,Horror,94,Spain,Spanish,Carlos Almón Muñoz,Carlos Almón Muñoz,"Paula Pier, Fernando Tato, Alfonso Romeo, Clar...",Three friends film a documentary about Joanna ...,2.9,3.8,6.0,15.0,3.2,3.0,https://www.imdb.com/title/tt4561016,SpainSpanishThree friends film a documentary a...
75004,tt4561318,Webcam,2015,"Comedy, Drama",110,Iceland,Icelandic,Sigurður Anton,Sigurður Anton,"Anna Hafþórsdóttir, Telma Huld Jóhannesdóttir,...",A college student begins working as a cam girl...,6.4,7.2,1.0,unavailable,8.6,6.2,https://www.imdb.com/title/tt4561318,IcelandIcelandicA college student begins worki...
75005,tt4561448,Prithvi,2010,Drama,134,India,Kannada,Jacob Varghese,unavailable,"Puneeth Rajkumar, Parvathy Thiruvothu, Sriniva...",Prithvi Kumar is an young IAS Officer who work...,7.6,8.2,1.0,unavailable,5.8,7.3,https://www.imdb.com/title/tt4561448,IndiaKannadaPrithvi Kumar is an young IAS Offi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85768,tt9906644,Manoharam,2019,"Comedy, Drama",122,India,Malayalam,Anvar Sadik,unavailable,"Vineeth Sreenivasan, Aparna Das, Basil Joseph,...",Manoharan is a poster artist struggling to fin...,6.8,7.2,9.0,1.0,7.3,6.7,https://www.imdb.com/title/tt9906644,IndiaMalayalamManoharan is a poster artist str...
85769,tt9908390,Le lion,2020,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,"Alexandre Coquelle, Matthieu Le Naour","Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,5.5,unavailable,4.0,6.0,5.3,https://www.imdb.com/title/tt9908390,"France, BelgiumFrenchA psychiatric hospital pa..."
85770,tt9911196,De Beentjes van Sint-Hildegard,2020,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,"Radek Bajgar, Herman Finkers","Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,7.9,6.0,4.0,7.6,7.8,https://www.imdb.com/title/tt9911196,"NetherlandsGerman, DutchA middle-aged veterina..."
85771,tt9914286,Sokagin Çocuklari,2019,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,"Ahmet Faik Akinci, Kasim Uçkan","Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",unavailable,6.4,9.4,unavailable,unavailable,4.0,3.1,https://www.imdb.com/title/tt9914286,"TurkeyTurkishunavailableAhmet Faik Akinci, Bel..."


In [12]:
cv1 = CountVectorizer() 
count_matrix1 = cv1.fit_transform(df1["description"])

In [13]:
recommender1 = pairwise_distances(count_matrix1, metric='cosine')

In [14]:
recommender1

array([[0.        , 0.9       , 0.73273876, ..., 0.8245884 , 0.84808909,
        0.90465374],
       [0.9       , 0.        , 0.94654775, ..., 0.9561471 , 1.        ,
        0.95232687],
       [0.73273876, 0.94654775, 0.        , ..., 0.78903675, 0.86466701,
        0.8725882 ],
       ...,
       [0.8245884 , 0.9561471 , 0.78903675, ..., 0.        , 0.80014798,
        0.7909395 ],
       [0.84808909, 1.        , 0.86466701, ..., 0.80014798, 0.        ,
        0.85515864],
       [0.90465374, 0.95232687, 0.8725882 , ..., 0.7909395 , 0.85515864,
        0.        ]])

In [17]:
cv2 = CountVectorizer() 
count_matrix2 = cv2.fit_transform(df2["description"])
cv3 = CountVectorizer() 
count_matrix3 = cv3.fit_transform(df3["description"])
cv4 = CountVectorizer() 
count_matrix4 = cv4.fit_transform(df4["description"])

In [18]:
recommender2 = pairwise_distances(count_matrix2, metric='cosine')
recommender3 = pairwise_distances(count_matrix3, metric='cosine')
recommender4 = pairwise_distances(count_matrix4, metric='cosine')

In [19]:
rec_df1 = pd.DataFrame(recommender1, index = df1['title'].str.title(), columns = df1['title'].str.title())
rec_df2 = pd.DataFrame(recommender2, index = df2['title'].str.title(), columns = df2['title'].str.title())
rec_df3 = pd.DataFrame(recommender3, index = df3['title'].str.title(), columns = df3['title'].str.title())
rec_df4 = pd.DataFrame(recommender4, index = df4['title'].str.title(), columns = df4['title'].str.title())

In [20]:
rec_df4

title,Red Zone - 22 Miglia Di Fuoco,Birdland,The Last Witch,Webcam,Prithvi,O Amor É Lindo... Porque Sim!,First Man On Mars,Il Cittadino Illustre,Dreamland,Oflu Hoca'Nin Sifresi,...,Jessie,La Reina De Los Lagartos,Enemy Lines,Ottam,Pengalila,Manoharam,Le Lion,De Beentjes Van Sint-Hildegard,Sokagin Çocuklari,La Vida Sense La Sara Amat
title,,,,,,,,,,,,,,,,,,,,,
Red Zone - 22 Miglia Di Fuoco,0.000000,0.906250,0.783494,0.950971,0.875000,0.899777,0.920050,0.791486,0.952754,0.843750,...,0.765421,0.878732,0.725000,0.838251,0.953171,0.860246,0.746000,0.855662,1.0,0.972392
Birdland,0.906250,0.000000,0.855662,0.852913,0.725000,0.565700,0.866750,0.791486,0.905509,0.875000,...,0.791486,0.909049,0.800000,0.757376,0.836097,0.888197,0.873000,0.783494,1.0,0.834353
The Last Witch,0.783494,0.855662,0.000000,1.000000,0.807550,0.897131,0.938454,0.759228,1.000000,0.879719,...,0.779293,0.976662,0.730570,0.709466,1.000000,0.870901,0.780029,0.833333,1.0,0.893737
Webcam,0.950971,0.852913,1.000000,0.000000,0.921554,0.790343,0.958188,0.959107,0.925875,0.852913,...,1.000000,1.000000,0.960777,1.000000,0.889792,1.000000,1.000000,0.886772,1.0,0.826741
Prithvi,0.875000,0.725000,0.807550,0.921554,0.000000,0.866369,0.786799,0.603823,1.000000,0.750000,...,0.687228,0.902986,0.720000,0.568669,0.868878,0.798754,0.796800,0.826795,1.0,0.867482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Manoharam,0.860246,0.888197,0.870901,1.000000,0.798754,0.880477,0.713961,0.696936,0.957742,0.748442,...,0.766874,0.728837,0.754033,0.831215,0.937172,0.000000,0.687621,0.709526,1.0,0.901227
Le Lion,0.746000,0.873000,0.780029,1.000000,0.796800,0.932116,0.810464,0.655742,0.951998,0.809500,...,0.682224,0.753584,0.695200,0.726104,0.928632,0.687621,0.000000,0.743368,1.0,0.887801
De Beentjes Van Sint-Hildegard,0.855662,0.783494,0.833333,0.886772,0.826795,0.845697,0.846136,0.729132,0.945446,0.819578,...,0.789325,0.824965,0.740192,0.782100,0.891852,0.709526,0.743368,0.000000,1.0,0.808727


In [21]:
rec_df4['Webcam'].sort_values()[1:11]

title
Shao Nian De Ni            0.676425
Nichinichi Kore Kôjitsu    0.685055
Il Libro Di Henry          0.685515
Thelma                     0.689119
Darker Than Night          0.703932
La Fille Au Bracelet       0.705826
Matchbox                   0.710586
Perempuan Tanah Jahanam    0.713554
Mia E Il Leone Bianco      0.713554
Spinster                   0.713749
Name: Webcam, dtype: float64

In [22]:
rec_df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, Miss Jerry to An Unremarkable Life
Columns: 25000 entries, Miss Jerry to An Unremarkable Life
dtypes: float64(25000)
memory usage: 4.7+ GB


In [23]:
rec1 = rec_df1.astype('float32')

In [25]:
rec1 = np.around(rec1, decimals = 2)

In [26]:
rec1.head()

title,Miss Jerry,The Story Of The Kelly Gang,Den Sorte Drøm,Cleopatra,L'Inferno,"From The Manger To The Cross; Or, Jesus Of Nazareth",Madame Dubarry,Quo Vadis?,Independenta Romaniei,Richard Iii,...,Le Trésor Des Îles Chiennes,La Fiammiferaia,Turner E Il «Casinaro»,Täcknamn Coq Rouge,Uhf - I Vidioti,Un Mondo Senza Pietà,Io E Zio Buck,Under The Boardwalk,Senza Un Colpo In Canna,An Unremarkable Life
title,,,,,,,,,,,,,,,,,,,,,
Miss Jerry,0.00,0.90,0.73,0.72,0.74,0.58,0.39,0.73,0.54,0.73,...,0.64,1.00,0.91,0.71,0.66,0.83,1.00,0.82,0.85,0.90
The Story Of The Kelly Gang,0.90,0.00,0.95,0.86,0.95,0.88,0.77,0.91,0.91,0.91,...,0.95,1.00,1.00,0.95,0.89,0.94,1.00,0.96,1.00,0.95
Den Sorte Drøm,0.73,0.95,0.00,0.81,0.80,0.73,0.65,0.85,0.76,0.76,...,0.75,0.92,0.95,0.82,0.79,0.88,0.87,0.79,0.86,0.87
Cleopatra,0.72,0.86,0.81,0.00,0.85,0.65,0.60,0.81,0.74,0.81,...,0.78,0.94,1.00,0.80,0.72,0.84,1.00,0.91,0.96,0.87
L'Inferno,0.74,0.95,0.80,0.85,0.00,0.65,0.66,0.81,0.76,0.76,...,0.67,0.92,1.00,0.75,0.76,0.80,0.81,0.70,0.87,0.97


In [27]:
rec1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, Miss Jerry to An Unremarkable Life
Columns: 25000 entries, Miss Jerry to An Unremarkable Life
dtypes: float32(25000)
memory usage: 2.3+ GB


In [29]:
rec1.to_csv('data/rec1.csv')

In [30]:
rec2 = rec_df2.astype('float32')
rec3 = rec_df3.astype('float32')
rec4 = rec_df4.astype('float32')

In [31]:
rec2 = np.around(rec2, decimals = 2)
rec3 = np.around(rec3, decimals = 2)
rec4 = np.around(rec4, decimals = 2)

In [32]:
rec2.to_csv('data/rec2.csv')
rec3.to_csv('data/rec3.csv')
rec4.to_csv('data/rec4.csv')

In [76]:
rec_df1.to_csv('data/final1.csv')
rec_df2.to_csv('data/final2.csv')
rec_df3.to_csv('data/final3.csv')
rec_df4.to_csv('data/final4.csv')

In [2]:
english = pd.read_csv('data/cleaned.csv')

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
english.head()

,imdb_title_id,title,year,genre,duration,country,language,director,writer,actors,description,weighted_average_vote,mean_vote,reviews_from_users,reviews_from_critics,females_allages_avg_vote,males_allages_avg_vote,link
0,tt0000009,Miss Jerry,1894,Romance,45,USA,None,Alexander Black,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,5.9,1.0,2.0,6.0,6.2,https://www.imdb.com/title/tt0000009
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,6.3,7.0,7.0,6.2,6.1,https://www.imdb.com/title/tt0000574
2,tt0001892,Den sorte drøm,1911,Drama,53,"Germany, Denmark",unavailable,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini","Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,6.0,5.0,2.0,5.7,5.9,https://www.imdb.com/title/tt0001892
3,tt0002101,Cleopatra,1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,5.3,25.0,3.0,5.9,5.1,https://www.imdb.com/title/tt0002101
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,6.9,31.0,14.0,7.2,7.0,https://www.imdb.com/title/tt0002130


In [4]:
import re

In [5]:
eng = english.where(english['language'].str.contains('English'))

In [6]:
eng

,imdb_title_id,title,year,genre,duration,country,language,director,writer,actors,description,weighted_average_vote,mean_vote,reviews_from_users,reviews_from_critics,females_allages_avg_vote,males_allages_avg_vote,link
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0002101,Cleopatra,1912,"Drama, History",100.0,USA,English,Charles L. Gaskill,Victorien Sardou,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,5.3,25.0,3.0,5.9,5.1,https://www.imdb.com/title/tt0002101
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
eng = eng.dropna()

In [8]:
eng

,imdb_title_id,title,year,genre,duration,country,language,director,writer,actors,description,weighted_average_vote,mean_vote,reviews_from_users,reviews_from_critics,females_allages_avg_vote,males_allages_avg_vote,link
3,tt0002101,Cleopatra,1912,"Drama, History",100.0,USA,English,Charles L. Gaskill,Victorien Sardou,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,5.3,25.0,3.0,5.9,5.1,https://www.imdb.com/title/tt0002101
5,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,"Biography, Drama",60.0,USA,English,Sidney Olcott,Gene Gauntier,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,5.8,13.0,5.0,5.8,5.7,https://www.imdb.com/title/tt0002199
9,tt0002461,Richard III,1912,Drama,55.0,"France, USA",English,"André Calmettes, James Keane","James Keane, William Shakespeare","Robert Gemp, Frederick Warde, Albert Gardner, ...",Richard of Gloucester uses manipulation and mu...,5.5,5.4,8.0,1.0,7.7,5.5,https://www.imdb.com/title/tt0002461
17,tt0003167,Amore di madre,1914,Drama,55.0,USA,English,D.W. Griffith,"D.W. Griffith, H.E. Aitken","Henry B. Walthall, Josephine Crowell, Lillian ...",John Howard Payne at his most miserable point ...,5.8,6.1,6.0,unavailable,6.4,5.7,https://www.imdb.com/title/tt0003167
18,tt0003419,Lo studente di Praga,1913,"Drama, Fantasy, Horror",85.0,Germany,"German, English","Paul Wegener, Stellan Rye","Hanns Heinz Ewers, Hanns Heinz Ewers","Paul Wegener, Grete Berger, Lyda Salmonova, Jo...","Balduin, a student of Prague, leaves his royst...",6.5,6.5,20.0,26.0,6.7,6.5,https://www.imdb.com/title/tt0003419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85756,tt9894470,VFW,2019,"Action, Crime, Horror",92.0,USA,English,Joe Begos,"Max Brallier, Matthew McArdle","Stephen Lang, William Sadler, Fred Williamson,...",A group of old war veterans put their lives on...,6.1,6.1,83.0,94.0,5.6,6.1,https://www.imdb.com/title/tt9894470
85757,tt9896916,The Pilgrim's Progress,2019,"Animation, Adventure, Family",108.0,USA,English,Robert Fernandez,"John Bunyan, Robert Fernandez","David Thorpe, John Rhys-Davies, Kristyn Getty,...","An epic journey, faithfully adapted to modern-...",5.7,7.0,28.0,3.0,6.9,5.2,https://www.imdb.com/title/tt9896916
85758,tt9898858,Coffee & Kareem,2020,"Action, Comedy",88.0,USA,English,Michael Dowse,Shane Mack,"Ed Helms, Taraji P. Henson, Terrence Little Ga...",Twelve-year-old Kareem Manning hires a crimina...,5.1,5.1,388.0,64.0,5.1,5.1,https://www.imdb.com/title/tt9898858
85760,tt9899880,Columbus,2018,"Comedy, Drama",82.0,Iran,"Persian, English",Hatef Alimardani,Hatef Alimardani,"Farhad Aslani, Majid Salehi, Saeed Poursamimi,...",A rich family are deciding to immigrate to the...,4.0,4.0,unavailable,13.0,3.5,4.1,https://www.imdb.com/title/tt9899880


In [9]:
features = ['country', 'language', 'description', 'actors']

In [10]:
def combine_features(row):
    return row['country'] + row['language'] + row['description'] + row['actors']

In [11]:
for feature in features:
    eng[feature] = eng[feature].fillna('')
    eng["combined_features"] = eng.apply(combine_features,axis=1)

<ipython-input-11-0fdab63837c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng[feature] = eng[feature].fillna('')
<ipython-input-11-0fdab63837c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng["combined_features"] = eng.apply(combine_features,axis=1)


In [12]:
eng

,imdb_title_id,title,year,genre,duration,country,language,director,writer,actors,description,weighted_average_vote,mean_vote,reviews_from_users,reviews_from_critics,females_allages_avg_vote,males_allages_avg_vote,link,combined_features
3,tt0002101,Cleopatra,1912,"Drama, History",100.0,USA,English,Charles L. Gaskill,Victorien Sardou,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,5.3,25.0,3.0,5.9,5.1,https://www.imdb.com/title/tt0002101,USAEnglishThe fabled queen of Egypt's affair w...
5,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,"Biography, Drama",60.0,USA,English,Sidney Olcott,Gene Gauntier,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,5.8,13.0,5.0,5.8,5.7,https://www.imdb.com/title/tt0002199,USAEnglishAn account of the life of Jesus Chri...
9,tt0002461,Richard III,1912,Drama,55.0,"France, USA",English,"André Calmettes, James Keane","James Keane, William Shakespeare","Robert Gemp, Frederick Warde, Albert Gardner, ...",Richard of Gloucester uses manipulation and mu...,5.5,5.4,8.0,1.0,7.7,5.5,https://www.imdb.com/title/tt0002461,"France, USAEnglishRichard of Gloucester uses m..."
17,tt0003167,Amore di madre,1914,Drama,55.0,USA,English,D.W. Griffith,"D.W. Griffith, H.E. Aitken","Henry B. Walthall, Josephine Crowell, Lillian ...",John Howard Payne at his most miserable point ...,5.8,6.1,6.0,unavailable,6.4,5.7,https://www.imdb.com/title/tt0003167,USAEnglishJohn Howard Payne at his most misera...
18,tt0003419,Lo studente di Praga,1913,"Drama, Fantasy, Horror",85.0,Germany,"German, English","Paul Wegener, Stellan Rye","Hanns Heinz Ewers, Hanns Heinz Ewers","Paul Wegener, Grete Berger, Lyda Salmonova, Jo...","Balduin, a student of Prague, leaves his royst...",6.5,6.5,20.0,26.0,6.7,6.5,https://www.imdb.com/title/tt0003419,"GermanyGerman, EnglishBalduin, a student of Pr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85756,tt9894470,VFW,2019,"Action, Crime, Horror",92.0,USA,English,Joe Begos,"Max Brallier, Matthew McArdle","Stephen Lang, William Sadler, Fred Williamson,...",A group of old war veterans put their lives on...,6.1,6.1,83.0,94.0,5.6,6.1,https://www.imdb.com/title/tt9894470,USAEnglishA group of old war veterans put thei...
85757,tt9896916,The Pilgrim's Progress,2019,"Animation, Adventure, Family",108.0,USA,English,Robert Fernandez,"John Bunyan, Robert Fernandez","David Thorpe, John Rhys-Davies, Kristyn Getty,...","An epic journey, faithfully adapted to modern-...",5.7,7.0,28.0,3.0,6.9,5.2,https://www.imdb.com/title/tt9896916,"USAEnglishAn epic journey, faithfully adapted ..."
85758,tt9898858,Coffee & Kareem,2020,"Action, Comedy",88.0,USA,English,Michael Dowse,Shane Mack,"Ed Helms, Taraji P. Henson, Terrence Little Ga...",Twelve-year-old Kareem Manning hires a crimina...,5.1,5.1,388.0,64.0,5.1,5.1,https://www.imdb.com/title/tt9898858,USAEnglishTwelve-year-old Kareem Manning hires...
85760,tt9899880,Columbus,2018,"Comedy, Drama",82.0,Iran,"Persian, English",Hatef Alimardani,Hatef Alimardani,"Farhad Aslani, Majid Salehi, Saeed Poursamimi,...",A rich family are deciding to immigrate to the...,4.0,4.0,unavailable,13.0,3.5,4.1,https://www.imdb.com/title/tt9899880,"IranPersian, EnglishA rich family are deciding..."


In [13]:
eng1 = eng.iloc[:15000, :]
eng2 = eng.iloc[15001:30000, :]
eng3 = eng.iloc[30001:, :]

In [14]:
cv = CountVectorizer()
matrix = cv.fit_transform(eng1['combined_features'])
cv = CountVectorizer()
matrix2 = cv.fit_transform(eng2['combined_features'])
cv = CountVectorizer()
matrix3 = cv.fit_transform(eng3['combined_features'])

In [15]:
matrix = matrix.astype('float32')
matrix2 = matrix2.astype('float32')
matrix3 = matrix3.astype('float32')

In [16]:
reco = pairwise_distances(matrix, metric='cosine')

In [17]:
reco2 = pairwise_distances(matrix2, metric='cosine')

In [18]:
reco3 = pairwise_distances(matrix3, metric='cosine')

In [19]:
reco_df = pd.DataFrame(reco, index = eng1['title'].str.title(), columns = eng1['title'].str.title())
reco_df2 = pd.DataFrame(reco2, index = eng2['title'].str.title(), columns = eng2['title'].str.title())
reco_df3 = pd.DataFrame(reco2, index = eng2['title'].str.title(), columns = eng2['title'].str.title())

In [20]:
r_df = np.around(reco_df, decimals = 2)

In [21]:
r_df2 = np.around(reco_df2, decimals = 2)

In [22]:
r_df3 = np.around(reco_df3, decimals = 2)

In [23]:
r_df.to_csv('data/eng1.csv')
r_df2.to_csv('data/eng2.csv')
r_df3.to_csv('data/eng3.csv')